In [1]:
# library
from pyspark import SparkContext, SparkConf
from pyspark import StorageLevel
from pyspark.sql.functions import * #col, to_date, unix_timestamp
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import numpy as np
import pandas as pd
import datetime as dt
import time
import shutil
from dateutil.relativedelta import relativedelta
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
sqlContext = SQLContext(sc)

from pyspark.sql.functions import *
from pyspark.sql.types import *
#from DatabaseModule import DatabaseModule
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta
from pyspark.sql import functions as F
#db=DatabaseModule()
## library
from pyspark import StorageLevel
from pyspark.sql.functions import * #col, to_date, unix_timestamp
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.window import Window
import numpy as np
import pandas as pd
import datetime as dt
import shutil
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
from operator import add

## Mias
#from pyspark.context import SparkContext
#from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, abs, sum # Solo me traigo tres funciones del paquete functions
from pyspark.sql import functions as F # Me traigo todas las funciones del paquete functions
from pyspark.sql.types import StringType, IntegerType
from numpy import array, ravel
#from sklearn.neural_network import MLPRegressor
from pyspark.mllib.stat import Statistics

def sqlread(txtsql):
    """
    Carga los datos de una sql
    """
    postgres_host = "poolpostgresxl.marathon.mesos:5432"  
    postgres_db = "mutua"                            # change this with the database you want to connect

    user = os.environ.get("JPY_USER")

    sslCert = spark.conf.get("spark.ssl.datastore.certPem.path")
    sslKey = spark.conf.get("spark.ssl.datastore.keyPKCS8.path")
    sslRootCert = spark.conf.get("spark.ssl.datastore.caPem.path")

    url = ("jdbc:postgresql://{host}/{db}?user={user}&ssl=true&sslmode=verify-full&sslcert={sslCert}&sslrootcert={sslRootCert}&sslkey={sslKey}"
                  .format(sslCert=sslCert,
                          sslKey=sslKey,
                          sslRootCert=sslRootCert,
                          user=user,
                          host=postgres_host,
                          db=postgres_db))


    options = {"driver":"org.postgresql.Driver"}
    rst = spark.read.format("jdbc").options(url=url,dbtable=txtsql,properties=options).load()
    return rst


def unionAll(*dfs):
    first, *rest = dfs  
    return sqlContext.createDataFrame(
        sc.union([df.rdd.repartition(50) for df in dfs]),
        first.schema
    )

def printC(df, active = 1 ):
    """
    Carga los datos de una sql
    """
    if active == 1:
        print(df.count())

In [4]:
sqlread("(\
select msid, nui, ip, ua, uaf, dvf, uafv, osfv, p, res, lg, cast('desc' as text) ingh, ts \
from mutua.autos_vt_main \
limit 10 \
) t").limit(10).show()

+--------+--------------------+---------------+--------------------+-------------+--------+--------------------+--------------+---+-----------+-----+----+--------------------+
|    msid|                 nui|             ip|                  ua|          uaf|     dvf|                uafv|          osfv|  p|        res|   lg|ingh|                  ts|
+--------+--------------------+---------------+--------------------+-------------+--------+--------------------+--------------+---+-----------+-----+----+--------------------+
|90277284|3976fd55-9d68-47b...|193.146.115.134|Mozilla/5.0 (Wind...|      Firefox|   Other|        Firefox/52.0|    Windows XP|web| '1440x900'|en-US|desc|2017-03-30 00:01:...|
|90277303|0179b5a1-b7d3-445...|   88.10.214.45|Mozilla/5.0 (Wind...|       Chrome|   Other|    Chrome/56.0.2924|     Windows 7|web| '1024x640'|   es|desc|2017-03-30 00:03:...|
|90277307|c05dd435-1d6a-460...|   71.6.150.144|Mozilla/5.0 (Wind...|      Firefox|   Other|        Firefox/23.0|     Win

In [10]:
sqlread("(\
select idyy, nsol, count(*) nnui \
from \
( \
  select date_part('year', ts) idyy, 'nui' tipo, nui, count(distinct msid) nsol \
  from mutua.autos_vt_main \
  group by date_part('year', ts), nui \
  union all \
  select date_part('year', ts) idyy, 'ip' tipo, ip, count(distinct msid) nsol \
  from mutua.autos_vt_main \
  group by date_part('year', ts), nui \
) t \
group by idyy, nsol \
) t").toPandas().to_csv("res1.txt")